In [51]:
#match_away_team_info table
from glob import glob
import pandas as pd
# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')

column_away= ['match_id', 'player_id','full_name']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_away]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [52]:
#matchhometeaminfo table
column_home= ['match_id', 'player_id', 'full_name']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [53]:
# Match_event_info table
files_event = []
column_event= ['match_id','winner_code','start_datetime']
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_event.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/event*.parquet')
    )
event_info = [pd.read_parquet(file, columns=column_event) for file in files_event]
event_info = pd.concat(event_info, ignore_index=True)

In [4]:
tour_info = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    tour_info.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/tournament*.parquet')
    )

df_tour_info = [pd.read_parquet(file) for file in tour_info]
df_tour_info = pd.concat(df_tour_info, ignore_index=True)

In [ ]:
import pandas as pd
import numpy as np

df = (
    event_info
    .merge(df_tour_info[['match_id', 'tournament_id']], on='match_id')
    .merge(df_home_team_info[['match_id', 'player_id', 'full_name']], on='match_id', suffixes=('', '_home'))
    .merge(df_away_team_info[['match_id', 'player_id', 'full_name']], on='match_id', suffixes=('_home', '_away'))
)

# --- 2. Identify the actual winner player_id ---
df['winner_id'] = np.where(df['winner_code'] == 1, df['player_id_home'],
                           np.where(df['winner_code'] == 2, df['player_id_away'], np.nan))

df['winner_name'] = np.where(df['winner_code'] == 1, df['full_name_home'],
                             np.where(df['winner_code'] == 2, df['full_name_away'], np.nan))

# --- 3. Convert start_datetime to month (using UNIX timestamp in seconds) ---
df['start_datetime'] = pd.to_datetime(df['start_datetime'], unit='s', errors='coerce')
df['month'] = df['start_datetime'].dt.to_period('M')  # e.g., '2023-07'

# --- 4. Count unique tournaments won by each player in each month ---
tournament_wins = (
    df.groupby(['winner_id', 'winner_name', 'month'])['tournament_id']
      .nunique()
      .reset_index(name='tournaments_won')
)

max_row = tournament_wins.loc[tournament_wins['tournaments_won'].idxmax()]

print(f"Player: {max_row['winner_name']} (ID: {max_row['winner_id']}) "
      f"won {max_row['tournaments_won']} tournaments in {max_row['month']}.")


Player: Naw, Hazem (ID: 197446.0) won 7 tournaments in 2024-02.
